In [50]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
import yaml
import os
import pandas as pd
import numpy as np
import seaborn as sns
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
import aequitas.plot as ap

In [186]:
datadir = os.path.join(os.getcwd(),'data/donors-choose/single-model-audit')

In [59]:
with open(os.path.join(datadir,'best_precision.yaml')) as file:
    metadata = yaml.load(file, Loader=yaml.FullLoader)

# Model with Highest Precision at top 1000

In [6]:
metadata

{'val_predictions': '',
 'test_predictions': 'predictions_c598fbe93f4c218ac7d325fb478598f1.csv.gz',
 'model_uuid': 'a04e2eedd9c5ff18bcf77e84ae9db561',
 'model_classpath': 'sklearn.ensemble.RandomForestClassifier',
 'model_hyperparameters': None,
 'training_set': {'matrix_uuid': '',
  'num_rows': 0,
  'num_cols': 114,
  'label_col': 'quickstart_label'},
 'eval_params': {'target_pp': {1000: {'global_threshold': 0.527624878013593}}}}

In [187]:
preds = pd.read_csv(os.path.join(datadir,metadata['test_predictions']), compression='gzip')

In [188]:
preds.shape

(17677, 4)

# Thresholding


In [189]:
preds.head()

,entity_id,as_of_date,quickstart_label,predict_proba
0,185730,1354320000000,0.0,0.308565
1,185729,1354320000000,0.0,0.371010
2,185738,1354320000000,0.0,0.438470
3,185737,1354320000000,0.0,0.470824
4,185736,1354320000000,0.0,0.248637


In [190]:
preds = preds.sort_values('predict_proba', ascending = False).reset_index(drop=True).copy()

preds['score'] = preds.apply(lambda x: 1.0 if int(x.name)  < 1000 else 0.0, axis=1)

In [191]:
preds['score'].value_counts()

0.0    16677
1.0     1000
Name: score, dtype: int64

In [192]:
preds.rename(columns={'quickstart_label':'label_value'}, inplace=True)

In [193]:
preds.head()

,entity_id,as_of_date,label_value,predict_proba,score
0,168836,1359504000000,0.0,0.658896,1.0
1,169208,1359331200000,1.0,0.657936,1.0
2,173882,1358121600000,1.0,0.657853,1.0
3,184207,1354665600000,0.0,0.652161,1.0
4,180129,1355529600000,1.0,0.651767,1.0


In [170]:
datadir = os.path.join(os.getcwd(),'data/donors-choose-splits/')
attr_df = pd.read_csv(os.path.join(datadir,'test_20121201_20130201_protected.csv.gz'), compression='gzip')
attr_df.tail(10)

,entity_id,as_of_date,poverty_level,teacher_sex,metro_type
17667,168445,1359590400000,highest,female,urban
17668,168446,1359590400000,highest,female,suburban_rural
17669,168447,1359590400000,lower,female,suburban_rural
17670,168448,1359590400000,highest,female,urban
17671,168449,1359590400000,highest,female,urban
17672,168451,1359590400000,lower,female,urban
17673,168452,1359590400000,lower,female,suburban_rural
17674,168453,1359590400000,lower,female,unknown
17675,168430,1359590400000,highest,female,suburban_rural
17676,168476,1359590400000,highest,female,urban


In [52]:
attr_df['poverty_']

(165618, 6)

In [194]:
df = pd.merge(preds[['entity_id','as_of_date','score','label_value']], attr_df, how='left', on=['entity_id','as_of_date'], left_index=True, right_index=False, sort=True, copy=True)

In [195]:
df.tail()

,entity_id,as_of_date,score,label_value,poverty_level,teacher_sex,metro_type
332,186059,1354320000000,0.0,0.0,highest,female,suburban_rural
333,186060,1354320000000,0.0,0.0,lower,female,suburban_rural
334,186061,1354320000000,0.0,0.0,lower,female,unknown
335,186062,1354320000000,0.0,0.0,lower,female,urban
337,186063,1354320000000,0.0,0.0,highest,female,urban


In [162]:
df['poverty_level'].value_counts()

highest    9973
lower      7704
Name: poverty_level, dtype: int64

In [164]:
df['metro_type'].value_counts()

urban             9115
suburban_rural    6506
unknown           2056
Name: metro_type, dtype: int64

In [163]:
df['teacher_sex'].value_counts()

female    15093
male       2584
Name: teacher_sex, dtype: int64

In [205]:
df['label_value'].sum() / len(df)

0.33772698987384736

## Aequitas

In [173]:
g = Group()
xtab, _ = g.get_crosstabs(df[['score','label_value','poverty_level','metro_type', 'teacher_sex']].copy())

model_id, score_thresholds 0 {'rank_abs': [1000]}


In [174]:
absolute_metrics = g.list_absolute_metrics(xtab)

In [175]:
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

,model_id,score_threshold,k,attribute_name,attribute_value,pp,pn,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities
0,0,binary 0/1,1000,poverty_level,highest,247,9726,89,2954,6772,158,3112,6861,9973,17677
1,0,binary 0/1,1000,poverty_level,lower,753,6951,359,2464,4487,394,2858,4846,7704,17677
2,0,binary 0/1,1000,metro_type,suburban_rural,626,5880,293,2060,3820,333,2393,4113,6506,17677
3,0,binary 0/1,1000,metro_type,unknown,182,1874,83,666,1208,99,765,1291,2056,17677
4,0,binary 0/1,1000,metro_type,urban,192,8923,72,2692,6231,120,2812,6303,9115,17677
5,0,binary 0/1,1000,teacher_sex,female,951,14142,424,4620,9522,527,5147,9946,15093,17677
6,0,binary 0/1,1000,teacher_sex,male,49,2535,24,798,1737,25,823,1761,2584,17677


In [176]:
xtab[['attribute_name', 'attribute_value'] + absolute_metrics]

,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,ppr,pprev,prev
0,poverty_level,highest,0.050771,0.987028,0.303722,0.360324,0.012972,0.949229,0.696278,0.639676,0.247,0.024767,0.312043
1,poverty_level,lower,0.137859,0.925918,0.354481,0.476760,0.074082,0.862141,0.645519,0.523240,0.753,0.097741,0.370976
2,metro_type,suburban_rural,0.139156,0.928762,0.350340,0.468051,0.071238,0.860844,0.649660,0.531949,0.626,0.096219,0.367814
3,metro_type,unknown,0.129412,0.935709,0.355390,0.456044,0.064291,0.870588,0.644610,0.543956,0.182,0.088521,0.372082
4,metro_type,urban,0.042674,0.988577,0.301692,0.375000,0.011423,0.957326,0.698308,0.625000,0.192,0.021064,0.308502
5,teacher_sex,female,0.102390,0.957370,0.326686,0.445846,0.042630,0.897610,0.673314,0.554154,0.951,0.063009,0.341019
6,teacher_sex,male,0.030377,0.986371,0.314793,0.489796,0.013629,0.969623,0.685207,0.510204,0.049,0.018963,0.318498


In [201]:
b = Bias()
bdf = b.get_disparity_predefined_groups(xtab, original_df=df, ref_groups_dict={'poverty_level':'lower', 'metro_type':'suburban_rural', 'teacher_sex':'male'})

get_disparity_predefined_group()


In [181]:
metrics = ['tpr']

In [202]:
ap.disparities(bdf, metrics, 'poverty_level', fairness_threshold = 1.30)

alt.LayerChart(...)

In [203]:
ap.disparities(bdf, metrics, 'metro_type', fairness_threshold = 1.30)

alt.LayerChart(...)

In [204]:
ap.disparities(bdf, metrics, 'teacher_sex', fairness_threshold = 1.30)

alt.LayerChart(...)